In [28]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import cv2 
import face_recognition
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Input, Conv2D, Flatten, Dense

In [6]:
img_size = (48, 48)
train_data_path = './images/train'
validation_data_path = './images/validation'

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

In [10]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=img_size,
    batch_size=64,
    color_mode='grayscale', 
    class_mode='categorical'
)

Found 28821 images belonging to 7 classes.


In [16]:
validation_generator = validation_datagen.flow_from_directory(
    validation_data_path,
    target_size=img_size,
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 7066 images belonging to 7 classes.


In [22]:
model = Sequential()

In [24]:
#pip install keras

Note: you may need to restart the kernel to use updated packages.


In [30]:
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

In [32]:
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

In [34]:
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

In [36]:
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax')) 

In [38]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [52]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=5,
    callbacks=[early_stopping]
)

Epoch 1/5
450/450 ━━━━━━━━━━━━━━━━━━━━ 53s 116ms/step - accuracy: 0.4495 - loss: 1.4259 - val_accuracy: 0.5220 - val_loss: 1.2687
Epoch 2/5
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 72us/step - accuracy: 0.5312 - loss: 1.2835 - val_accuracy: 0.6538 - val_loss: 1.1820
Epoch 3/5
450/450 ━━━━━━━━━━━━━━━━━━━━ 58s 129ms/step - accuracy: 0.4601 - loss: 1.3929 - val_accuracy: 0.4786 - val_loss: 1.4012
Epoch 4/5
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 74us/step - accuracy: 0.4688 - loss: 1.4605 - val_accuracy: 0.4231 - val_loss: 1.5190
Epoch 5/5
450/450 ━━━━━━━━━━━━━━━━━━━━ 54s 120ms/step - accuracy: 0.4790 - loss: 1.3670 - val_accuracy: 0.5061 - val_loss: 1.2994


In [56]:
model.save('my_model.keras')

In [60]:
loaded_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [62]:
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [64]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:

    ret, frame = cap.read()

    if not ret:
        break

In [ ]:
face_locations = face_recognition.face_locations(frame)

In [ ]:
 for face_location in face_locations:
        
        top, right, bottom, left = face_location

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 255), 2)

        
        face_image = frame[top:bottom, left:right]
        gray_face = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
        gray_face = cv2.resize(gray_face, (48, 48), interpolation=cv2.INTER_AREA)

        
        roi = gray_face.astype('float') / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        preds = model.predict(roi)[0]
        emotion_label = emotion_labels[preds.argmax()]

        cv2.putText(frame, emotion_label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

In [ ]:
cv2.imshow('Emotion Detector', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()